In [ ]:
using SemidefiniteOptInterface
const SDOI = SemidefiniteOptInterface

using Test

using MathOptInterface
const MOI = MathOptInterface
const MOIT = MOI.Test
const MOIB = MOI.Bridges
const MOIU = MOI.Utilities

We hack the `optimize!` call of CSDP to print the `mock_optimize!` function to be used in the tests for the mock_optimizer.
The output still needs to be edited and replaced by the exact solution but this is a good start

In [ ]:
const MOIU = MOI.Utilities
MOIU.@model(SDModelData,
    (), (MOI.EqualTo, MOI.GreaterThan, MOI.LessThan),
    (MOI.Zeros, MOI.Nonnegatives, MOI.Nonpositives, MOI.PositiveSemidefiniteConeTriangle), (),
    (MOI.SingleVariable,), (MOI.ScalarAffineFunction,),
    (MOI.VectorOfVariables,), ())

In [ ]:
import CSDP
optimizer = CSDP.Optimizer(printlevel=0)

cache = MOIU.UniversalFallback(SDModelData{Float64}())
cached = MOIU.CachingOptimizer(cache, optimizer)
bridged = MOIB.full_bridge_optimizer(cached, Float64)

config = MOIT.TestConfig(atol=1e-4, rtol=1e-4)

In [ ]:
function MOI.optimize!(m::SDOI.SOItoMOIBridge)
    MOI.optimize!(m.sdoptimizer)
    X = SDOI.getX(m.sdoptimizer)
    println("MOIU.set_mock_optimize!(mock, (mock) -> MOIU.mock_optimize!(mock,")
    print(  "                                                            ")
    if MOI.get(m, MOI.DualStatus()) == MOI.INFEASIBILITY_CERTIFICATE
        print("tuple()")
    else
        if MOI.get(m, MOI.PrimalStatus()) == MOI.INFEASIBILITY_CERTIFICATE
            print("(MOI.InfeasibilityCertificate, ")
        end
        print(  "[")
        for (blk, dim) in enumerate(m.blockdims)
            if blk > 1
                print(", ")
            end
            #print(round.(SDOI.block(X, blk), 4))
            print(SDOI.block(X, blk))
        end
        print("]")
        if MOI.get(m, MOI.PrimalStatus()) == MOI.INFEASIBILITY_CERTIFICATE
            print(")")
        end
    end
    if MOI.get(m, MOI.PrimalStatus()) != MOI.INFEASIBILITY_CERTIFICATE
        println(",")
        print(  "                                                            ")
        #print(round.(SDOI.gety(m.sdoptimizer), 4))
        print(SDOI.gety(m.sdoptimizer))
    end
    println("))")
end

In [ ]:
function generate_mock_test(testfun)
    testfun(bridged, config)
    testcall = replace(string(testfun), "MathOptInterface.Test" => "MOIT")
    println("$testcall(bridged, config)")
end

In [ ]:
generate_mock_test(MOIT.psdt0ftest)

In [ ]:
for testfun in map(p -> p.second, sort(collect(MOIT.lintests), by=p->p.first))
    generate_mock_test(testfun)
end

In [ ]:
mock = SDOI.MockSDOptimizer{Float64}()
mock_optimizer = SDOI.SDOIOptimizer(mock, Float64)
cached = MOIU.CachingOptimizer(SDModelData{Float64}(), mock_optimizer)
bridged = MOIB.full_bridge_optimizer(cached, Float64)
config = MOIT.TestConfig(atol=1e-4, rtol=1e-4)